In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, ResFlow, NVPFlow, phase, Normal
import numpy as np
from matplotlib import pyplot as plt

np.random.seed(1234)

In [2]:
n = 20
s1 = 0.1
m1 = 0.
dim = [3,4]

params = []
params.append(np.random.normal(size=dim))
for i in range(n-1):
    new = params[i] + np.random.normal(loc=m1, scale=s1, size=dim)
    params.append(new)
params = np.array(params)

PWalk = NormalRW(dim[0]*dim[1], sigma=s1, mu=m1)

In [3]:
params.shape

(20, 3, 4)

In [4]:
def autoregr(X, param):
    d = param[:,:3]
    X = np.matmul(d, X) + param[:,-1] + np.random.normal(size=[3], scale=0.2)
    return X

def autoregr_tf(X, param):
    d = param[:,:3]
    X = tf.matmul(d, X) + param[:,-1]
    return X

In [5]:
xs = [np.random.normal(size=dim[0])]
for i in range(n-1):
    xs.append(autoregr(xs[i], params[i]))
xs = np.array(xs)[np.newaxis,:].astype('float32')

In [6]:
def create_step_flow(name=None, prev_flow=None):
    with tf.variable_scope(name, reuse=None):
        if prev_flow is not None:
            aux_vars = prev_flow.output
        else:
            aux_vars = None
            
        step_flow = DFlow([ResFlow(dim=dim[0]*dim[1], name='res{}'.format(i), aux_vars=aux_vars)\
                           for i in range(4)])
    return step_flow

In [7]:
flows = [create_step_flow('step_flow0')]
for i in range(n-1):
    new = create_step_flow('step_flow' + str(i+1), flows[-1])
    flows.append(new)

In [8]:
outputs = [x.output for x in flows]
outputs_ = [outputs[0]]
for i in range(len(outputs)-1):
    outputs_.append(outputs[i+1] + outputs_[-1])
outputs = outputs_
outputs = tf.concat(outputs, axis=0)[tf.newaxis]

In [9]:
prior = PWalk.logdens(outputs)

In [10]:
#outputs = tf.cast(outputs, tf.float64)

In [11]:
prior

<tf.Tensor 'add_19:0' shape=() dtype=float32>

In [12]:
def create_loglik():
    obs_d = Normal(dim=None, sigma=0.2, mu=0)
    out = tf.reshape(outputs, [n, 3, 4])
    
    ll = 0
    for i in range(n-1):
        pred = xs[0,i+1] - autoregr_tf(xs[0,i][:,np.newaxis], out[i])
        ll += obs_d.logdens(pred)
    return ll

In [13]:
logl = create_loglik()

In [14]:
xs.shape

(1, 20, 3)

In [15]:
logl

<tf.Tensor 'add_57:0' shape=() dtype=float32>

In [16]:
ent = sum([flow.logdens for flow in flows])

In [17]:
ent

<tf.Tensor 'add_77:0' shape=(1,) dtype=float32>

In [18]:
loss = -logl - prior + ent
loss /= 20*dim[0]*dim[1]*1000

In [19]:
opt = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [20]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
for _ in range(100):
    for _ in range(200):
        l, _ = sess.run([loss, opt], {phase:True})
    print(l)

[2110032.]
[5867716.5]
[2251188.]
[5771058.5]
[2517589.5]
[2461424.]
[4805402.]
[2159194.]
[3264644.2]
[2180989.]
[3777115.2]
[2406082.2]
[3265285.]
[1563479.]
[1286560.8]
[1047077.5]
[4997999.5]
[5956709.]
[3413319.2]
[2189512.]
[1954879.4]
[1907580.2]
[986314.9]
[2182118.5]
[1910403.8]
[1504573.9]
[548547.1]
[2538513.2]
[1505601.4]
[404607.3]
[1556417.6]
[926984.75]
[361005.5]
[706467.7]
[725137.5]
[1016254.6]
[792714.8]
[938099.94]
[3424351.5]
[1210319.2]
[328797.25]
[1063259.8]
[295961.22]
[526643.25]
[1361205.1]
[620953.3]
[342363.8]
[791543.75]
[979197.06]
[291149.28]
[614848.9]
[579217.5]
[778326.8]
[482521.53]
[616390.5]
[592434.25]
[549081.2]
[1264852.6]
[619095.9]
[152962.7]
[276045.25]
[175809.9]
[362043.44]
[807127.56]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
[nan]
